<a href="https://colab.research.google.com/github/Aathisakthi-G/project_1_cls/blob/main/project_1_cls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **cuda**

In [ ]:
import torch
print(torch.cuda.is_available())


# **mount drive**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **modules for data coversion**

In [ ]:
!pip install ultralytics
!pip install pycocotools


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.6/913.6 kB 47.0 MB/s eta 0:00:00


# **code for conversion**

In [ ]:
import os
import json
from pathlib import Path

def coco_to_yolo_segmentation(coco_json_path, images_dir, output_labels_dir):
    """
    Convert COCO JSON annotations to YOLOv8 segmentation format.
    Args:
        coco_json_path (str): Path to the COCO JSON file.
        images_dir (str): Path to the images directory.
        output_labels_dir (str): Path to the output labels directory.
    """
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # Prepare category mapping (COCO category ID -> YOLO class index)
    category_mapping = {category['id']: idx for idx, category in enumerate(coco_data['categories'])}

    # Process annotations
    for image in coco_data['images']:
        image_id = image['id']
        image_filename = image['file_name']

        # Read image dimensions
        img_width, img_height = image['width'], image['height']

        # Find all annotations for this image
        annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == image_id]

        # Prepare label file path in the output directory
        label_path = os.path.join(output_labels_dir, f"{Path(image_filename).stem}.txt")
        os.makedirs(output_labels_dir, exist_ok=True)

        with open(label_path, 'w') as label_file:
            for ann in annotations:
                category_id = ann['category_id']
                class_idx = category_mapping[category_id]
                segmentation = ann['segmentation'][0]  # Assuming one segmentation per annotation

                # Normalize segmentation points to YOLO format
                normalized_seg = [
                    point / dim for point, dim in zip(segmentation, [img_width, img_height] * (len(segmentation) // 2))
                ]
                normalized_seg_str = " ".join(map(str, normalized_seg))
                label_file.write(f"{class_idx} {normalized_seg_str}\n")

    print(f"Annotations for {coco_json_path} converted and saved to {output_labels_dir}")

# Define the paths for your dataset
base_dir = '/content/drive/MyDrive/splitted_dataset'
output_base_dir = '/content/drive/MyDrive/YOLOv8_converted'  # New directory for converted annotations
splits = ['test','train', 'val']

# Convert COCO annotations to YOLOv8 format for each split
for split in splits:
    coco_json_path = f"{base_dir}/{split}/{split}_annotations.json"  # Path to COCO JSON for each split
    images_dir = f"{base_dir}/{split}/images"
    output_labels_dir = f"{output_base_dir}/{split}/labels"  # New directory for labels

    coco_to_yolo_segmentation(coco_json_path, images_dir, output_labels_dir)

print("COCO annotations successfully converted to YOLOv8 format and stored separately!")


Annotations for /content/drive/MyDrive/splitted_dataset/test/test_annotations.json converted and saved to /content/drive/MyDrive/YOLOv8_converted/test/labels
Annotations for /content/drive/MyDrive/splitted_dataset/train/train_annotations.json converted and saved to /content/drive/MyDrive/YOLOv8_converted/train/labels
Annotations for /content/drive/MyDrive/splitted_dataset/val/val_annotations.json converted and saved to /content/drive/MyDrive/YOLOv8_converted/val/labels
COCO annotations successfully converted to YOLOv8 format and stored separately!


# **creating yamal**

In [ ]:
import os
import json
import yaml

# Path to the COCO JSON file (example for train annotations)
coco_json_path = input("Enter the full path of your COCO JSON file: ")  # Example: '/content/drive/MyDrive/splitted_dataset/train/train_annotations.json'

# Load the COCO JSON file
with open(coco_json_path, 'r') as f:
    coco_data = json.load(f)

# Extract class names
class_names = [category['name'] for category in coco_data['categories']]

# Prompt for output directory
output_base_dir = input("Enter the base output directory for the converted dataset (e.g., '/content/drive/MyDrive/YOLOv8_converted/'): ")

# Define the directory structure for the YAML file
dataset_yaml = {
    'train': os.path.join(output_base_dir, 'train/images'),
    'val': os.path.join(output_base_dir, 'val/images'),
    'test': os.path.join(output_base_dir, 'test/images'),
    'nc': len(class_names),  # Number of classes
    'names': class_names  # List of class names
}

# Path to save the YAML file
yaml_output_path = os.path.join(output_base_dir, 'dataset.yaml')

# Save the YAML file to the specified directory
with open(yaml_output_path, 'w') as yaml_file:
    yaml.dump(dataset_yaml, yaml_file)

# Print the path of the saved YAML file
print(f"YAML file has been saved to: {yaml_output_path}")


Enter the full path of your COCO JSON file: /content/drive/MyDrive/splitted_dataset/OFF_AUG.json
Enter the base output directory for the converted dataset (e.g., '/content/drive/MyDrive/YOLOv8_converted/'): /content/drive/MyDrive/YOLOv8_converted
YAML file has been saved to: /content/drive/MyDrive/YOLOv8_converted/dataset.yaml


# **train yolov8n-seg**

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n-seg.pt")
model.train(
    data="/content/drive/MyDrive/YOLOv8_converted/dataset.yaml",
    epochs=50,
    imgsz=640
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.74M/6.74M [00:00<00:00, 276MB/s]

Ultralytics 8.3.68 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/drive/MyDrive/YOLOv8_converted/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=F

100%|██████████| 755k/755k [00:00<00:00, 129MB/s]


Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 296MB/s]


AMP: checks passed ✅


train: Scanning /content/drive/MyDrive/YOLOv8_converted/train/labels... 57 images, 7 backgrounds, 3 corrupt:   1%|          | 64/6476 [01:44<2:54:56,  1.64s/it]


KeyboardInterrupt: 